# Frame the problem and look at the big picture
This is exercise #4 from Chapter 3 of Hands-On Machine Learning, which covers classification. 

1. __Define the objective.__ The task is to build a spam classifier using data from [Apache SpamAssassin's public datasets](https://spamassassin.apache.org/old/publiccorpus/) ([README](https://spamassassin.apache.org/old/publiccorpus/readme.html)). 

2. __How will your solution be used?__ This could be used by an email service like Gmail to automatically flag and filter out messages that have a high probability of being spam. 

3. __What are the current solutions/workarounds (if any)?__ Well, spam filters do already exist in most if not all email services. That being said, some spam emails still get through, so it's up to users to identify those on their own. In addition, when a spam email does get through, users can mark it as spam, and all future emails from that address will be automatically marked as spam. 

4. __How should you frame this problem (supervised/unsupervised, online/offline, etc.)?__ This is a __supervised classification task__ because we are making binary predictions (spam vs. ham) and we have labeled training data. Ideally, this is an online learning task, because spammers never stop thinking of new ways to trick people, so the model should be learning from new data constantly. However, for the sake of this exercise, it will be an offline system using static training data. 

5. __How should performance be measured?__ This is a bit more complex than the Titanic task, for example, where _accuracy_ was our primary performance metric. Is it better to lean further towards spam and risk misclassifying safe emails as spam? Or is it better to lean further towards "ham" and risk missing spam emails?
    - According to ChatGPT: "For a spam classifier, it is generally more fitting to __prioritize precision over recall__." This is because a spam classifier that marks safe emails as spam is a real nuisance to users, whereas missing a few spam emails is not as big a deal. By focusing on precision, we will minimize false positives and reduce the chances of misclassifying safe emails. 
    
    - `Precision = TP / (TP + FP)`
    
    - That being said, __there should also be a minimum recall.__ I could predict just one instance as spam, and if I'm correct, that means my precision is 100%. However, recall would be way too low.  
    
6. __Is the performance measure aligned with the business objective?__ Yes. Optimizing for precision will reduce the number of false positives, and therefore create a better hypothetical user experience. Since the spam classifier would most likely be used in a commercial product, user experience is the most important thing. 

7. __What would be the minimum performance needed to reach the business objective?__ It seems that a __precision of 95%__ is the widely recommended benchmark for spam classifiers. Regarding minimum recall, maybe let's see baseline performance of different models to estimate what a realistic minimum might be. 

8. __What are comparable problems? Can you reuse experience or tools?__ Beyond the Titanic dataset, this is really my first end-to-end classification project. From my work on the [MNIST dataset](https://github.com/iherman10/mnist-classification/blob/main/chapter_3.ipynb), I can reuse some charting function to analyze performance and compare models. From my work on the [Titanic dataset](https://github.com/iherman10/titanic/blob/main/titanic_2.ipynb), I can reuse the custom transformer class structures for data transformation purposes.

9. __Is human expertise available?__ The internet :) I'm working on this solo. 

10. __How would you solve the problem manually?__ When I try to eyeball whether an email is spam or not, I consider:
    - Have I received emails from this sender before?
    - Are there obvious typos?
    - Are there links? 
    - Are they asking for money? Or credit card information? Etc. 
    - Are they writing in all caps? Or all lower-case? 
    
    These considerations might influence the feature engineering part of this project. 

11. __List assumptions you've made so far.__ 
    - I should be able to identify most spam just by looking at it. 
    
    - I'll have to leverage text transformations to process the data. 
    
    - Baseline models should get me most of the way towards my performance goal, and feature engineering/hyperparameter tuning will get me the rest of the way.  

12. __Verify assumptions if possible.__ TBD...

In [10]:
# Libraries 
import pandas as pd 
import numpy as np 

import os
import tarfile 
import urllib.request

import joblib

import email 
import email.policy

# Get the data
Available data from SpamAssassin website: 
- `spam`: 500 spam messages 
- `spam_2`: 1396 spam messages, added more recently. 
- `easy_ham`: 2500 non-spam messages, fairly easy to differentiate.  
- `easy_ham_2`: 144 non-spam messages, added more recently. 
- `hard_ham`: 250 non-spam messages, harder to differentiate. 

In total, there are __6046__ messages with about a 31% spam ratio. 

In [2]:
# Download data 
DOWNLOAD_ROOT = 'http://spamassassin.apache.org/old/publiccorpus/'

SPAM_URL = DOWNLOAD_ROOT + '20030228_spam.tar.bz2'
SPAM_2_URL = DOWNLOAD_ROOT + '20050311_spam_2.tar.bz2'
EASY_HAM_URL = DOWNLOAD_ROOT + '20030228_easy_ham.tar.bz2'
EASY_HAM_2_URL = DOWNLOAD_ROOT + '20030228_easy_ham_2.tar.bz2'
HARD_HAM_URL = DOWNLOAD_ROOT + '20030228_hard_ham.tar.bz2'

SPAM_PATH = os.path.join('datasets', 'spam')

def fetch_spam_data(spam_url=SPAM_URL, 
                    spam_2_url=SPAM_2_URL, 
                    easy_ham_url=EASY_HAM_URL, 
                    easy_ham_2_url=EASY_HAM_2_URL, 
                    hard_ham_url=HARD_HAM_URL,  
                    spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (('spam.tar.bz2', SPAM_URL), 
                          ('spam_2.tar.bz2', SPAM_2_URL), 
                          ('easy_ham.tar.bz2', EASY_HAM_URL), 
                          ('easy_ham_2.tar.bz2', EASY_HAM_2_URL), 
                          ('hard_ham.tar.bz2', HARD_HAM_URL)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()

In [3]:
fetch_spam_data()

In [4]:
# Load all the emails 
SPAM_DIR = os.path.join(SPAM_PATH, 'spam')
SPAM_2_DIR = os.path.join(SPAM_PATH, 'spam_2')
EASY_HAM_DIR = os.path.join(SPAM_PATH, 'easy_ham')
EASY_HAM_2_DIR = os.path.join(SPAM_PATH, 'easy_ham_2')
HARD_HAM_DIR = os.path.join(SPAM_PATH, 'hard_ham')

spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]
spam_2_filenames = [name for name in sorted(os.listdir(SPAM_2_DIR)) if len(name) > 20]
easy_ham_filenames = [name for name in sorted(os.listdir(EASY_HAM_DIR)) if len(name) > 20]
easy_ham_2_filenames = [name for name in sorted(os.listdir(EASY_HAM_2_DIR)) if len(name) > 20]
hard_ham_filenames = [name for name in sorted(os.listdir(HARD_HAM_DIR)) if len(name) > 20]

In [9]:
print(f"""
spam: {len(spam_filenames)} files
spam_2: {len(spam_2_filenames)} files
easy_ham: {len(easy_ham_filenames)} files
easy_ham_2: {len(easy_ham_2_filenames)} files 
hard_ham: {len(hard_ham_filenames)} files 
""")


spam: 500 files
spam_2: 1396 files
easy_ham: 2500 files
easy_ham_2: 1400 files 
hard_ham: 250 files 



In [16]:
# Parse emails 
def load_email(directory, filename, spam_path=SPAM_PATH):
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
spam_emails = [load_email(directory='spam', filename=name) for name in spam_filenames]
spam_2_emails = [load_email(directory='spam_2', filename=name) for name in spam_2_filenames]
easy_ham_emails = [load_email(directory='easy_ham', filename=name) for name in easy_ham_filenames]
easy_ham_2_emails = [load_email(directory='easy_ham_2', filename=name) for name in easy_ham_2_filenames]
hard_ham_emails = [load_email(directory='hard_ham', filename=name) for name in hard_ham_filenames]

# Explore the data
Let's start by examining examples of ham vs. spam to understand what the data looks like.

In [23]:
print(easy_ham_emails[0].get_content().strip())

Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 

# Prepare the data

# Shortlist promising models

# Fine-tune the system

# Present your solution